#### Basic libraries imports

In [4]:
import pandas as pd
import csv
from datetime import datetime

#### Import and filter stations around Zurich HB

In [5]:
stops = pd.read_csv('../gtfsfp20192019-06-05/stops.txt', sep=',')

In [6]:
def filter_around_zurich(gps):
    #stop_id 8503000
    zurich_lon, zurich_lat = gps[gps['stop_id'] == '8503000'][['stop_lon', 'stop_lat']].values[0]
    radius = 10
    
    def distance_between_gps(lon1, lat1, lon2, lat2):
    #https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
        from math import sin, cos, sqrt, atan2, radians

        R = 6373.0

        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c

        return distance

    def inside_radius(row):
        return distance_between_gps(zurich_lon, zurich_lat, row['stop_lon'], row['stop_lat']) <= radius

    return gps[gps.apply(inside_radius, axis=1)]

In [7]:
zurich_stops = filter_around_zurich(stops)
zurich_stops_set = set(zurich_stops['stop_id'])
len(zurich_stops_set)

1213

#### Import and filter connections
The connections have been created by downloading the [SBB timetable 2019](https://opentransportdata.swiss/de/dataset/timetable-2019-gtfs) and converting the stop_times to connections with the help of the npm package [gtfs2lc](https://github.com/linkedconnections/gtfs2lc).

In [8]:
timestamp_2019_06_17 = datetime.timestamp(datetime.strptime('2019-06-17', '%Y-%m-%d'))


with open('../gtfsfp20192019-06-05/connexioni.csv') as f_in:
    with open('../gtfsfp20192019-06-05/zurich_connexioni.csv', 'w', newline='') as f_out:
        counter = 0
        csv_reader = csv.reader(f_in, delimiter=',')
        csv_writer = csv.writer(f_out, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for row in csv_reader:
            if counter == 0:
                # write header
                csv_writer.writerow(row)
            # if departureStop (row[1]) and arrivalStop (row[3]) are within the Zurich zone, keep line.
            if row[1] in zurich_stops_set and row[3] in zurich_stops_set:
                # transform datetimes to timestamps
                departure_timestamp = int(datetime.timestamp(datetime.strptime(row[2], '%Y-%m-%dT%H:%M:%S.%fZ'))) + 7200
                arrival_timestamp = int(datetime.timestamp(datetime.strptime(row[4], '%Y-%m-%dT%H:%M:%S.%fZ'))) + 7200
                row[2] = departure_timestamp
                row[4] = arrival_timestamp
                # simplify stations (maybe change back later?)
                row[1] = row[1][:row[1].index(':')] if ':' in row[1] else row[1]
                row[3] = row[3][:row[3].index(':')] if ':' in row[3] else row[3]
                csv_writer.writerow(row)
            if counter % 100000 == 0:
                print(counter)
            counter += 1
print("finished")

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [44]:
zurich_connections = pd.read_csv('../gtfsfp20192019-06-05/zurich_connexioni.csv')
# reduce to one week timeframe
zurich_connections = zurich_connections[zurich_connections['departureTime'] >= timestamp_2019_06_17]
zurich_connections = zurich_connections[zurich_connections['departureTime'] < timestamp_2019_06_17 + 691200]
zurich_connections = zurich_connections.sort_values(by=['departureTime', 'arrivalTime'])
zurich_connections.head()

,id,departureStop,departureTime,arrivalStop,arrivalTime,trip,route,headsign
3205019,23797628,8503000,1560722400,8503003,1560722520,532.TA.26-6-A-j19-1.76.R,26-6-A-j19-1,Zürich Tiefenbrunnen
2775364,20113272,8503006,1560722400,8503129,1560722580,40.TA.26-8-A-j19-1.5.R,26-8-A-j19-1,Effretikon
3711579,28388290,8503020,1560722460,8503001,1560722700,724.TA.26-11-j19-1.192.R,26-11-j19-1,Aarau
2882007,21144158,8503003,1560722520,8503000,1560722700,444.TA.26-5-A-j19-1.151.H,26-5-A-j19-1,Zug
3089494,23039421,8503127,1560722520,8503128,1560722700,505.TA.26-14-j19-1.28.R,26-14-j19-1,Zürich HB


#### Write the sorted Zurich-Connections dataframe to disk.
For now, use same format as in [Captain Train implementation](https://github.com/trainline-eu/csa-challenge/blob/master/csa.py)

In [45]:
zurich_lean = zurich_connections.loc[:,['departureStop', 'arrivalStop', 'departureTime', 'arrivalTime', 'trip']]
zurich_lean.to_csv(path_or_buf='../gtfsfp20192019-06-05/zurich_lean.csv', sep=' ', index=False, header=False)

In [54]:
zurich_stops.to_csv(path_or_buf='../gtfsfp20192019-06-05/zurich_stops.csv', sep=',', index=False)

#### Filter trips by zurich location

In [20]:
zurich_trips = set(zurich_lean['trip'].values)
len(zurich_trips)
zurich_trips = set(zurich_lean['trip'].values)
trips_df = pd.read_csv('../gtfsfp20192019-06-05/trips.txt', sep=',')
zurich_trips_df = trips_df[trips_df['trip_id'].isin(zurich_trips)]
zurich_trips_df.head()
#zurich_trips_df.to_csv(path_or_buf='../gtfsfp20192019-06-05/zurich_trips.csv', sep=',', index=False)

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id
28,26-145-j19-1,TA+b0gmk,1000.TA.26-145-j19-1.5.R,"Thalwil, Zentrum",3386,1
32,26-15-A-j19-1,TA+b0nnr,1000.TA.26-15-A-j19-1.9.R,"Zürich, Bucheggplatz",30990,1
34,26-17-A-j19-1,TA+b0ewy,1000.TA.26-17-A-j19-1.23.R,"Zürich, Werdhölzli",30869,1
49,26-31-j19-1,TA+b074b,1000.TA.26-31-j19-1.16.H,"Zürich Altstetten, Bahnhof",19752,0
50,26-32-j19-1,TA+b0091,1000.TA.26-32-j19-1.11.R,"Zürich, Holzerhurd",31980,1
